### Initial setup

In [4]:
import cv2
import os
import json
from utils import *

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Lego detection

In [5]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 0

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# ----------------------------------------------

grabcut_mask = grab_cut(image)
grabcut = cv2.bitwise_and(image, image, mask=grabcut_mask)

bg_median_mask = background_median(hsv)
bg_median = cv2.bitwise_and(image, image, mask=bg_median_mask)

canny_mask = canny_edges(gray)
canny = cv2.bitwise_and(image, image, mask=canny_mask)

combined_mask = cv2.bitwise_or(bg_median_mask, canny_mask)
final_mask = cv2.bitwise_and(grabcut_mask, combined_mask)
result = cv2.bitwise_and(image, image, mask=final_mask)

# ----------------------------------------------

contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = filter_contours(contours)
for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (255, 255, 0), 2)

num_lego_pieces = len(contours)
print("Number of LEGO pieces detected:", num_lego_pieces)

# ----------------------------------------------

cv2.imshow('Image', image)
cv2.imshow('GrabCut', grabcut)
cv2.imshow('Background Median', bg_median)
cv2.imshow('Canny', canny)
cv2.imshow('Mask', result)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------

Number of LEGO pieces detected: 1
